In [397]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb 
from xgboost import XGBClassifier, plot_importance
from itertools import combinations


### 전처리1. df_test에서 모든 컬럼이 WT인 컬럼들 df와 df_test에서 삭제

In [398]:
df = pd.read_csv('../input/train.csv')
#df.drop(columns=['IDH1', 'TP53'], inplace=True)
df_test = pd.read_csv('../input/test.csv')
df_test.drop(columns='ID', inplace=True)
#df_test.drop(columns=['IDH1', 'TP53'], inplace=True)

cdf = df_test.apply(lambda x : x!='WT').agg(sum).reset_index(name='count')  #.sort_values(by='count', ascending=False)
to_remove = cdf[cdf['count'] == 0 ]['index'].values
df.drop(columns = to_remove, axis=1, inplace=True)
df_test.drop(columns = to_remove, axis=1, inplace=True)
df.shape
data = df.drop(columns=['ID', 'SUBCLASS'])
disease_list = df.SUBCLASS.unique()

In [399]:
data = df.drop('ID', axis=1)


In [ ]:

total = pd.concat([data, df_test])
data.iloc[:, 1:] = data.iloc[:, 1:].applymap(lambda x : 0 if x =='WT' else 1)
data

In [ ]:
from itertools import combinations
data2 = data.drop('SUBCLASS',axis=1)

results = []
#컬럼 수
num_columns = df_test.shape[1]

# 10개씩 나누어 접근하기
for i in range(0, num_columns, 10):
    # 각 범위의 시작 및 끝 인덱스 계산
    print("first_range", i, i + 10)  # 첫 번째 10개 컬럼
    data2_1 = data2.iloc[:, i: i+10]
    column_pairs = combinations(data2_1.columns, 2)

    for colA, colB in column_pairs:
        #if colA.startswith('m_'): break;    
        if not (colA in data2.columns) or not (colB in data2.columns):
            break;
        # 데이터프레임에서 colA가 'WT'가 아닌 샘플의 'SUBCLASS' 값 카운트
        colA_distribution = pd.Series(df_test[colA].apply(lambda x: 1 if x != 'WT' else 0))
        colB_distribution = pd.Series(df_test[colB].apply(lambda x: 1 if x != 'WT' else 0))
        
        cosine_sim = cosine_similarity(colA_distribution.values.reshape(1, -1), colB_distribution.values.reshape(1, -1))[0][0]
        
        if cosine_sim > 0.90:
            print(colA, colB, cosine_sim)
        #    results.append((colA, colB, cosine_sim))
#results_df = pd.DataFrame(results, columns=['Column 1', 'Column 2', 'Cosine Similarity'])

first_range 0 10
first_range 10 20
first_range 20 30
first_range 30 40
first_range 40 50
first_range 50 60
first_range 60 70
first_range 70 80
first_range 80 90
first_range 90 100
first_range 100 110
first_range 110 120
first_range 120 130
first_range 130 140
first_range 140 150
first_range 150 160
first_range 160 170
first_range 170 180
first_range 180 190
first_range 190 200
first_range 200 210
first_range 210 220
first_range 220 230
first_range 230 240
first_range 240 250
first_range 250 260
first_range 260 270
first_range 270 280
first_range 280 290
first_range 290 300
first_range 300 310
first_range 310 320
first_range 320 330
first_range 330 340
first_range 340 350
first_range 350 360
first_range 360 370
first_range 370 380
first_range 380 390
first_range 390 400
first_range 400 410
first_range 410 420
first_range 420 430
first_range 430 440
first_range 440 450
first_range 450 460
first_range 460 470
first_range 470 480
first_range 480 490
first_range 490 500
first_range 500 510


In [252]:
from itertools import combinations
data2 = data.drop('SUBCLASS',axis=1)

results = []
#컬럼 수
num_columns = data2.shape[1]

# 10개씩 나누어 접근하기
for i in range(0, num_columns, 10):
    # 각 범위의 시작 및 끝 인덱스 계산
    print("first_range", i, i + 10)  # 첫 번째 10개 컬럼
    data2_1 = data2.iloc[:, i: i+10]
    column_pairs = combinations(data2_1.columns, 2)

    for colA, colB in column_pairs:
        #if colA.startswith('m_'): break;    
        if not (colA in data2.columns) or not (colB in data2.columns):
            break;
        # 데이터프레임에서 colA가 'WT'가 아닌 샘플의 'SUBCLASS' 값 카운트
        colA_distribution = df[df[colA].apply(lambda x: x != 'WT')]['SUBCLASS'].value_counts()
        colB_distribution = df[df[colB].apply(lambda x: x != 'WT')]['SUBCLASS'].value_counts()

        # 두 분포의 인덱스를 맞추기
        combined_distribution = pd.DataFrame({
            colA: colA_distribution,
            colB: colB_distribution
        }).fillna(0)  # NaN 값을 0으로 채움
        if combined_distribution.empty or combined_distribution.shape[1] == 0:
            print("No data available for cosine similarity calculation.") 
            break
        # 코사인 유사도
        cosine_sim = cosine_similarity(combined_distribution.T)[0][1]
        if cosine_sim > 0.92:
            results.append((colA, colB, cosine_sim))
            #merge
            new_colname = "m_"+colA+"_"+colB
            total[new_colname] =  np.where(total[colA] != 'WT', total[colA], total[colB])
            #drop
            total.drop(columns=[colA, colB], inplace=True)
            data2.drop(columns=[colA, colB], inplace=True)
    
    print("second_range", i + 10 - 3, i + 10 + 3)  # 두 번째 10개 컬럼 (3개 이동)
    data2_2 = data2.iloc[:, i + 10 - 3:  i + 10 + 3]
    column_pairs2 = combinations(data2_2.columns, 2)
    
    for colA, colB in column_pairs2:
        #if colA.startswith('m_'): break;
        if not (colA in data2.columns) or not (colB in data2.columns):
            break;
        colA_distribution = df[df[colA].apply(lambda x: x != 'WT')]['SUBCLASS'].value_counts()
        colB_distribution = df[df[colB].apply(lambda x: x != 'WT')]['SUBCLASS'].value_counts()

        # 두 분포의 인덱스를 맞추기
        combined_distribution = pd.DataFrame({
            colA: colA_distribution,
            colB: colB_distribution
        }).fillna(0)  # NaN 값을 0으로 채움
        if combined_distribution.empty or combined_distribution.shape[1] == 0:
            print("No data available for cosine similarity calculation.") 
            break

        # 코사인 유사도
        cosine_sim = cosine_similarity(combined_distribution.T)[0][1]
        if cosine_sim > 0.92:
            results.append((colA, colB, cosine_sim))
            #merge
            new_colname = "m_"+colA+"_"+colB
            total[new_colname] =  np.where(total[colA] != 'WT', total[colA], total[colB])
            #drop
            total.drop(columns=[colA, colB], inplace=True)
            data2.drop(columns=[colA, colB], inplace=True)

results_df = pd.DataFrame(results, columns=['Column 1', 'Column 2', 'Cosine Similarity'])


first_range 0 10
second_range 7 13
first_range 10 20
second_range 17 23
first_range 20 30
second_range 27 33
first_range 30 40
second_range 37 43
first_range 40 50
second_range 47 53
first_range 50 60
second_range 57 63
first_range 60 70
second_range 67 73
first_range 70 80
second_range 77 83
first_range 80 90
second_range 87 93
first_range 90 100
No data available for cosine similarity calculation.
second_range 97 103
first_range 100 110
second_range 107 113
first_range 110 120
second_range 117 123
first_range 120 130
second_range 127 133
first_range 130 140
second_range 137 143
first_range 140 150
second_range 147 153
first_range 150 160
second_range 157 163
first_range 160 170
second_range 167 173
first_range 170 180
second_range 177 183
first_range 180 190
second_range 187 193
first_range 190 200
second_range 197 203
first_range 200 210
second_range 207 213
first_range 210 220
second_range 217 223
first_range 220 230
second_range 227 233
first_range 230 240
second_range 237 243
fir

In [255]:
total.drop('SUBCLASS',inplace=True, axis=1)

In [256]:
total.to_csv('total_92.csv',index=False)

In [257]:
data2

,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA3,ABCA4,ABCA8,ABCA9,ABCB1,...,ZNF112,ZNF277,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZW10,ZWINT
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [240]:
df

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,TRAIN_6196,LUAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,TRAIN_6197,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,TRAIN_6198,COAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,TRAIN_6199,TGCT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [258]:
4360-3672

688

In [259]:
4360-4016

344

In [260]:
set(total.columns).difference(data2.columns)

{'m_A2M_ABCA2',
 'm_ABCA5_ABCA6',
 'm_ABCB11_ABCC3',
 'm_ABCC4_ABCC5',
 'm_ABCD3_ABI1',
 'm_ABI3BP_ABLIM1',
 'm_ACE_ACO2',
 'm_ACOX2_ACP1',
 'm_ACSL5_ACSM1',
 'm_ACVRL1_ADAD1',
 'm_ADAM12_ADAM15',
 'm_ADAM23_ADCY1',
 'm_ADCY6_ADD3',
 'm_ADD2_ADH7',
 'm_AFAP1L2_AFF1',
 'm_AGL_AGO4',
 'm_AGXT_AIFM3',
 'm_AKR1B10_AKR1C3',
 'm_AKT2_ALAS2',
 'm_ALDH1A2_ALDH1L1',
 'm_ALDOB_ALOX12B',
 'm_ALMS1_ALS2',
 'm_AMPD3_ANK1',
 'm_ANKH_ANLN',
 'm_ANO1_ANPEP',
 'm_AOC3_AOX1',
 'm_AP2M1_AP3B1',
 'm_APLP1_APLP2',
 'm_ARFGEF1_ARFGEF2',
 'm_ARHGEF11_ARHGEF12',
 'm_ARHGEF6_ARHGEF7',
 'm_AR_ARAF',
 'm_ASB7_ASF1A',
 'm_ATG4A_ATP1A3',
 'm_ATP2A1_ATP2B1',
 'm_ATP6V0A1_ATP6V0A4',
 'm_ATP6V1H_ATP8B1',
 'm_ATXN1_AUTS2',
 'm_AXIN1_AXL',
 'm_B3GNT3_B4GALNT2',
 'm_BAIAP2_BANK1',
 'm_BCAN_BCAR1',
 'm_BIN1_BIRC3',
 'm_BMPR1B_BMPR2',
 'm_BRCA1_BRCA2',
 'm_BUB1_BUB1B',
 'm_C1QC_C1R',
 'm_C5AR1_C8A',
 'm_CA4_CA6',
 'm_CACNA1A_CACNA1B',
 'm_CADM3_CALB2',
 'm_CALCRL_CALML5',
 'm_CALU_CAMK1D',
 'm_CAP2_CAPN9',
 'm_CCDC28A_CCK

In [261]:
total

,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA3,ABCA4,ABCA8,ABCA9,ABCB1,...,m_USP8_UTP20,m_VAV2_VCAN,m_VWF_WDR33,m_WFS1_WNK4,m_WSB1_WWC1,m_YBX2_YTHDC1,m_ZBTB17_ZC3H12A,m_ZEB1_ZFPM2,m_ZMIZ1_ZNF185,m_ZPBP_ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,R698K,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,Y904H,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,WT,WT,WT,WT,WT,WT,WT,WT,T86S,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2542,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2543,WT,WT,WT,WT,L217I,R5M,G606D,P1444L P1439L P1383L P1404L P970L,P752H,WT,...,P337H P260H,P163L,C898C,P19P,W38X,N102N,WT,E999D E1049D E1050D E1067D E1052D E992D E978D ...,D1051Tfs,N252I N251I
2544,WT,WT,WT,WT,WT,WT,R1517H,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [271]:
total = pd.read_csv('total_92.csv')
total

,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA3,ABCA4,ABCA8,ABCA9,ABCB1,...,m_USP8_UTP20,m_VAV2_VCAN,m_VWF_WDR33,m_WFS1_WNK4,m_WSB1_WWC1,m_YBX2_YTHDC1,m_ZBTB17_ZC3H12A,m_ZEB1_ZFPM2,m_ZMIZ1_ZNF185,m_ZPBP_ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,R698K,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,Y904H,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8742,WT,WT,WT,WT,WT,WT,WT,WT,T86S,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
8743,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
8744,WT,WT,WT,WT,L217I,R5M,G606D,P1444L P1439L P1383L P1404L P970L,P752H,WT,...,P337H P260H,P163L,C898C,P19P,W38X,N102N,WT,E999D E1049D E1050D E1067D E1052D E992D E978D ...,D1051Tfs,N252I N251I
8745,WT,WT,WT,WT,WT,WT,R1517H,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [213]:
import numpy as np
# 병합할 두 개의 컬럼이 있다고 가정
col1 = 'PARP9'
col2 = 'PARP2'
col3 = 'PARP12'
col4 = 'PARP14'

# 두 컬럼을 병합하여 하나의 새로운 컬럼 생성 (값이 없는 경우 다른 컬럼의 값 사용)
data['merged_parp2_9'] =  np.where(df[col1] != 'WT', df[col1], df[col2])
data['merged_parp12_14'] =  np.where(df[col1] != 'WT', df[col3], df[col4])

# 기존의 두 컬럼 삭제
# df.drop([col1, col2,col3,col4], axis=1, inplace=True)

# 병합된 컬럼 출력
#print(.head())


In [ ]:
import numpy as np
import pandas as pd

# 예시 데이터프레임 생성
data = {
    'col1': ['WT', 'A', 'B', 'C'],
    'col2': ['X', 'WT', 'Y', 'Z'],
    'col3': ['WT', 'M', 'N', 'O']
}
df = pd.DataFrame(data)

# 세 컬럼을 병합하여 하나의 새로운 컬럼 생성
df['merged_column'] = np.where(df['col1'] != 'WT', df['col1'], 
                                np.where(df['col2'] != 'WT', df['col2'], 
                                        df['col3']))

print(df)


In [159]:
# Get the index of the 'PARP2' column
parp2_index = df.columns.get_loc('PARP2')

# Show the result (adding 1 to convert from 0-based index to 1-based index)
parp2_index + 1

2828

In [ ]:
df.iloc[:,2828]

In [ ]:
import pandas as pd

# 데이터 불러오기
file_path = '/mnt/data/train_sample.csv'
df = pd.read_csv(file_path)

# 컬럼 수
num_columns = df.shape[1]

# 10개씩 나누어 접근하기
for i in range(0, num_columns, 10):
    # 현재 인덱스의 시작과 끝
    start_index = i
    end_index = min(i + 10, num_columns)  # 마지막 인덱스는 컬럼 수를 초과하지 않도록 설정
    
    # 선택된 컬럼 출력
    print(f"Columns from {start_index} to {end_index - 1}:")
    print(df.iloc[:, start_index:end_index].head(), "\n")  # 처음 몇 개의 행을 출력


In [148]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr, chisquare
from scipy.special import kl_div

# 변수 설정
colA = 'PARP2'
colB = 'PARP9'

# 데이터프레임에서 colA가 'WT'가 아닌 샘플의 'SUBCLASS' 값 카운트
colA_distribution = df[df[colA].apply(lambda x: x != 'WT')]['SUBCLASS'].value_counts()

# 데이터프레임에서 colB가 'WT'가 아닌 샘플의 'SUBCLASS' 값 카운트
colB_distribution = df[df[colB].apply(lambda x: x != 'WT')]['SUBCLASS'].value_counts()

# 두 분포의 인덱스를 맞추기
combined_distribution = pd.DataFrame({
    colA: colA_distribution,
    colB: colB_distribution
}).fillna(0)  # NaN 값을 0으로 채움

# 1. 코사인 유사도
cosine_sim = cosine_similarity(combined_distribution.T)[0][1]

# 2. 피어슨 상관계수
pearson_corr, _ = pearsonr(combined_distribution[colA], combined_distribution[colB])

# 3. 지니 계수 (Gini Coefficient)
def gini_coefficient(data):
    n = len(data)
    if n == 0:
        return 0
    return (np.sum(np.abs(data[:, None] - data)) / (2 * n * np.mean(data)))

gini_colA = gini_coefficient(combined_distribution[colA].values)
gini_colB = gini_coefficient(combined_distribution[colB].values)

# 4. 킬리안 거리 (Jensen-Shannon Divergence)
def jensen_shannon(p, q):
    p = p / np.sum(p)
    q = q / np.sum(q)
    m = 0.5 * (p + q)
    return 0.5 * (kl_div(p, m).sum() + kl_div(q, m).sum())

js_divergence = jensen_shannon(combined_distribution[colA].values, combined_distribution[colB].values)

# 5. 카이제곱 통계
# 각 분포의 값을 정리하여 0으로 채운 후 두 배열의 길이를 맞추기

# 관측 빈도 및 기대 빈도 배열로 변환
observed = combined_distribution[colA].values.astype(float)
expected = combined_distribution[colB].values.astype(float)

# 두 배열의 총합을 맞추기 위해 빈도 배열 조정
total_observed = np.sum(observed)
total_expected = np.sum(expected)

if total_observed != total_expected:
    # 비율을 계산하여 총합이 동일하게 맞추기
    expected = (expected / total_expected) * total_observed

# 카이제곱 통계 계산
chi2_stat, p_value = chisquare(observed, expected)

# 결과 출력
print(f"Cosine Similarity: {cosine_sim}")
print(f"Pearson Correlation: {pearson_corr}")
print(f"Gini Coefficient ({colA}): {gini_colA}, Gini Coefficient ({colB}): {gini_colB}")
print(f"Jensen-Shannon Divergence: {js_divergence}")
print(f"Chi-Squared Statistic: {chi2_stat}, p-value: {p_value}")

Cosine Similarity: 0.922253258678133
Pearson Correlation: 0.8571265306075808
Gini Coefficient (PARP2): 11.108108108108109, Gini Coefficient (PARP9): 11.058823529411764
Jensen-Shannon Divergence: 0.09587715485571699
Chi-Squared Statistic: inf, p-value: 0.0


In [152]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr, chisquare
from scipy.special import kl_div

# 변수 설정
colA = 'PARP2'
colB = 'PARP14'

# 데이터프레임에서 colA가 'WT'가 아닌 샘플의 'SUBCLASS' 값 카운트
colA_distribution = df[df[colA].apply(lambda x: x != 'WT')]['SUBCLASS'].value_counts()

# 데이터프레임에서 colB가 'WT'가 아닌 샘플의 'SUBCLASS' 값 카운트
colB_distribution = df[df[colB].apply(lambda x: x != 'WT')]['SUBCLASS'].value_counts()

# 두 분포의 인덱스를 맞추기
combined_distribution = pd.DataFrame({
    colA: colA_distribution,
    colB: colB_distribution
}).fillna(0)  # NaN 값을 0으로 채움

# 1. 코사인 유사도
cosine_sim = cosine_similarity(combined_distribution.T)[0][1]

# 2. 피어슨 상관계수
pearson_corr, _ = pearsonr(combined_distribution[colA], combined_distribution[colB])

# 3. 지니 계수 (Gini Coefficient)
def gini_coefficient(data):
    n = len(data)
    if n == 0:
        return 0
    return (np.sum(np.abs(data[:, None] - data)) / (2 * n * np.mean(data)))

gini_colA = gini_coefficient(combined_distribution[colA].values)
gini_colB = gini_coefficient(combined_distribution[colB].values)

# 4. 킬리안 거리 (Jensen-Shannon Divergence)
def jensen_shannon(p, q):
    p = p / np.sum(p)
    q = q / np.sum(q)
    m = 0.5 * (p + q)
    return 0.5 * (kl_div(p, m).sum() + kl_div(q, m).sum())

js_divergence = jensen_shannon(combined_distribution[colA].values, combined_distribution[colB].values)

# 5. 카이제곱 통계
# 각 분포의 값을 정리하여 0으로 채운 후 두 배열의 길이를 맞추기

# 관측 빈도 및 기대 빈도 배열로 변환
observed = combined_distribution[colA].values.astype(float)
expected = combined_distribution[colB].values.astype(float)

# 두 배열의 총합을 맞추기 위해 빈도 배열 조정
total_observed = np.sum(observed)
total_expected = np.sum(expected)

if total_observed != total_expected:
    # 비율을 계산하여 총합이 동일하게 맞추기
    expected = (expected / total_expected) * total_observed

# 카이제곱 통계 계산
chi2_stat, p_value = chisquare(observed, expected)

# 결과 출력
print(f"Cosine Similarity: {cosine_sim}")
print(f"Pearson Correlation: {pearson_corr}")
print(f"Gini Coefficient ({colA}): {gini_colA}, Gini Coefficient ({colB}): {gini_colB}")
print(f"Jensen-Shannon Divergence: {js_divergence}")
print(f"Chi-Squared Statistic: {chi2_stat}, p-value: {p_value}")

Cosine Similarity: 0.8887673641445807
Pearson Correlation: 0.8016396097635758
Gini Coefficient (PARP2): 13.108108108108109, Gini Coefficient (PARP14): 9.598290598290598
Jensen-Shannon Divergence: 0.09615339052888061
Chi-Squared Statistic: 17.808891664774016, p-value: 0.6610639269067848


In [ ]:
from joblib import Parallel, delayed

# 병렬로 상관계수 계산
def calculate_corr(df_part):
    return df_part.corr()

# 데이터를 나누어 병렬로 상관계수 계산 수행
n_jobs = -1  # CPU 코어 수에 맞게 조정
correlation_matrices = Parallel(n_jobs=n_jobs)(delayed(calculate_corr)(df.iloc[:, i:i+1000]) for i in range(0, df.shape[1], 1000))

# 계산된 상관계수 행렬을 조합하여 사용


In [67]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
df = data 
# 1. 변동성이 적은 피처 제거
def remove_low_variance_features(df, threshold=0.95):
    non_variance_cols = [col for col in df.columns if df[col].value_counts(normalize=True).values[0] > threshold]
    df_reduced = df.drop(columns=non_variance_cols)
    return df_reduced

# 2. 상관계수 계산 후 유사한 피처 쌍 찾기 (임계값 이상인 컬럼 쌍 반환)
def find_similar_columns(correlation_matrix, threshold=0.95):
    similar_columns = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            col1 = correlation_matrix.columns[i]
            col2 = correlation_matrix.columns[j]
            corr_value = correlation_matrix.iloc[i, j]
            if abs(corr_value) > threshold:
                similar_columns.append((col1, col2))
    return similar_columns

# 3. 유사한 피처들 병합 (WT가 아닌 값을 우선하여 병합)
def merge_similar_columns(df, col_pairs):
    for col1, col2 in col_pairs:
        if col1 in df.columns and col2 in df.columns:
            # WT가 아닌 값을 우선적으로 선택하여 병합
            df[col1 + '_merged'] = np.where(df[col1] != 'WT', df[col1], df[col2])
            # 기존의 두 컬럼 삭제
            df.drop([col1, col2], axis=1, inplace=True)
    return df

# 4. 병렬로 상관계수 계산
def calculate_corr(df_part):
    return df_part.corr()

# 5. 통합된 실행 함수
def process_large_dataframe(df, sample_size=1000, corr_threshold=0.95, variance_threshold=0.95, chunk_size=1000):
    # 5.1. 변동성이 적은 피처 제거
    df_reduced = remove_low_variance_features(df, variance_threshold)
    
    # 5.2. 데이터 샘플링 (원본 데이터의 일부만 사용하여 상관계수 계산)
    sample_df = df_reduced.sample(n=sample_size, random_state=42)

    # 5.3. 병렬로 상관계수 계산 (데이터를 나누어 계산)
    n_jobs = -1  # CPU 코어 수에 맞게 조정
    feature_groups = [sample_df.iloc[:, i:i+chunk_size] for i in range(0, sample_df.shape[1], chunk_size)]
    correlation_matrices = Parallel(n_jobs=n_jobs)(delayed(calculate_corr)(group) for group in feature_groups)

    # 5.4. 상관계수가 높은 피처 쌍 찾기 및 병합
    for corr_matrix in correlation_matrices:
        similar_columns = find_similar_columns(corr_matrix, corr_threshold)
        df_reduced = merge_similar_columns(df_reduced, similar_columns)

    return df_reduced

# 데이터프레임 예시 (실제로는 df = pd.read_csv('your_dataframe.csv')로 로드)
# df = pd.read_csv('your_large_dataframe.csv')

# 4,000개 이상의 피처를 가진 데이터프레임 처리
processed_df = process_large_dataframe(df, sample_size=1000, corr_threshold=0.95, variance_threshold=0.95, chunk_size=1000)

# 결과 출력
print(processed_df.head())


   AHNAK  APC  ATRX  BRAF  COL6A3  DMD  DST  IDH1  KMT2D  MXRA5  ...  PKHD1  \
0      0    0     0     0       0    0    0     0      0      0  ...      0   
1      0    0     1     0       0    0    0     0      0      0  ...      0   
2      0    0     0     0       0    0    0     1      0      0  ...      0   
3      0    0     0     0       0    0    0     0      0      0  ...      0   
4      1    0     0     0       0    0    1     0      0      0  ...      0   

   PLEC  PTEN  RELN  RYR1  RYR2  SPTA1  SYNE1  TP53  VHL  
0     0     0     0     0     0      0      0     0    0  
1     0     0     0     0     0      0      0     1    0  
2     0     0     1     1     0      0      0     0    0  
3     0     0     0     0     0      0      0     0    0  
4     0     1     0     0     0      0      1     1    0  

[5 rows x 23 columns]


In [60]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

# 1. 변동성이 적은 피처 제거
def remove_low_variance_features(df, threshold=0.95):
    non_variance_cols = [col for col in df.columns if df[col].value_counts(normalize=True).values[0] > threshold]
    df_reduced = df.drop(columns=non_variance_cols)
    return df_reduced

# 2. 상관계수 계산 후 유사한 피처 쌍 찾기 (임계값 이상인 컬럼 쌍 반환)
def find_similar_columns(correlation_matrix, threshold=0.95):
    similar_columns = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            col1 = correlation_matrix.columns[i]
            col2 = correlation_matrix.columns[j]
            corr_value = correlation_matrix.iloc[i, j]
            if abs(corr_value) > threshold:
                similar_columns.append((col1, col2))
    return similar_columns

# 3. 유사한 피처들 병합 (WT가 아닌 값을 우선하여 병합, 기존 컬럼 유지)
def merge_similar_columns(df, col_pairs):
    for col1, col2 in col_pairs:
        if col1 in df.columns and col2 in df.columns:
            # WT가 아닌 값을 우선적으로 선택하여 병합 (새로운 병합된 컬럼 생성)
            merged_col_name = col1 + '_' + col2 + '_merged'
            df[merged_col_name] = np.where(df[col1] != 'WT', df[col1], df[col2])
    return df

# 4. 병렬로 상관계수 계산
def calculate_corr(df_part):
    return df_part.corr()

# 5. 통합된 실행 함수
def process_large_dataframe(df, sample_size=1000, corr_threshold=0.95, variance_threshold=0.95, chunk_size=1000):
    # 5.1. 변동성이 적은 피처 제거
    df_reduced = remove_low_variance_features(df, variance_threshold)
    
    # 5.2. 데이터 샘플링 (원본 데이터의 일부만 사용하여 상관계수 계산)
    sample_df = df_reduced.sample(n=sample_size, random_state=42)

    # 5.3. 병렬로 상관계수 계산 (데이터를 나누어 계산)
    n_jobs = -1  # CPU 코어 수에 맞게 조정
    feature_groups = [sample_df.iloc[:, i:i+chunk_size] for i in range(0, sample_df.shape[1], chunk_size)]
    correlation_matrices = Parallel(n_jobs=n_jobs)(delayed(calculate_corr)(group) for group in feature_groups)

    # 5.4. 상관계수가 높은 피처 쌍 찾기 및 병합
    for corr_matrix in correlation_matrices:
        similar_columns = find_similar_columns(corr_matrix, corr_threshold)
        df = merge_similar_columns(df, similar_columns)

    return df

# 데이터프레임 예시 (실제로는 df = pd.read_csv('your_dataframe.csv')로 로드)
# df = pd.read_csv('your_large_dataframe.csv')

# 4,000개 이상의 피처를 가진 데이터프레임 처리
processed_df = process_large_dataframe(df, sample_size=1000, corr_threshold=0.95, variance_threshold=0.95, chunk_size=1000)

# 결과 출력
print(processed_df.head())



   A2M  AAAS  AADAT  AARS1  ABAT  ABCA1  ABCA2  ABCA3  ABCA4  ABCA5  ...  \
0    0     0      0      0     0      0      0      0      0      0  ...   
1    0     0      0      0     0      0      0      0      0      0  ...   
2    1     0      0      0     0      0      0      0      0      0  ...   
3    0     0      0      0     0      0      0      0      0      0  ...   
4    0     0      0      0     0      0      0      0      0      0  ...   

   ZNF292  ZNF365  ZNF639  ZNF707  ZNFX1  ZNRF4  ZPBP  ZW10  ZWINT  ZYX  
0       0       0       0       0      0      0     0     0      0    0  
1       0       0       0       0      0      0     0     0      0    0  
2       0       0       0       0      0      0     0     0      0    0  
3       0       0       0       0      0      0     0     0      0    0  
4       0       0       0       0      0      0     0     0      0    0  

[5 rows x 4358 columns]


### 전처리2. total에서, 컬럼명+cell값 첫글자로 data Labeling 1차 변환, 이후 2차에서 Label Encoder XX

In [272]:
def process_cell(value, col_name):
    if value == 'WT':  # 값이 'WT'이면 빈 문자열
        return ''
    elif isinstance(value, str):  # 문자열일 때는 첫 글자 + 컬럼명
        return  value[0]+ col_name
    elif isinstance(value, (int, float)):  # 숫자일 때는 첫 숫자(소수점 전) + 컬럼명
        return  str(value)[0] + col_name
    return value  # 그 외에는 원래 값 반환

# 각 셀을 처리하는 함수 적용
for col in total.columns:
    total[col] = total[col].apply(lambda x: process_cell(x, col))


In [274]:
total

,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA3,ABCA4,ABCA8,ABCA9,ABCB1,...,m_USP8_UTP20,m_VAV2_VCAN,m_VWF_WDR33,m_WFS1_WNK4,m_WSB1_WWC1,m_YBX2_YTHDC1,m_ZBTB17_ZC3H12A,m_ZEB1_ZFPM2,m_ZMIZ1_ZNF185,m_ZPBP_ZYX
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,Rm_VWF_WDR33,,,,,,,
3,,,,,,,,,,,...,,,,,,,,Ym_ZEB1_ZFPM2,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8742,,,,,,,,,TABCA9,,...,,,,,,,,,,
8743,,,,,,,,,,,...,,,,,,,,,,
8744,,,,,LABCA1,RABCA3,GABCA4,PABCA8,PABCA9,,...,Pm_USP8_UTP20,Pm_VAV2_VCAN,Cm_VWF_WDR33,Pm_WFS1_WNK4,Wm_WSB1_WWC1,Nm_YBX2_YTHDC1,,Em_ZEB1_ZFPM2,Dm_ZMIZ1_ZNF185,Nm_ZPBP_ZYX
8745,,,,,,,RABCA4,,,,...,,,,,,,,,,


In [276]:
nRows = df.shape[0]
data = total[:nRows]
df_test = total[nRows:]

In [277]:
data

,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA3,ABCA4,ABCA8,ABCA9,ABCB1,...,m_USP8_UTP20,m_VAV2_VCAN,m_VWF_WDR33,m_WFS1_WNK4,m_WSB1_WWC1,m_YBX2_YTHDC1,m_ZBTB17_ZC3H12A,m_ZEB1_ZFPM2,m_ZMIZ1_ZNF185,m_ZPBP_ZYX
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,Rm_VWF_WDR33,,,,,,,
3,,,,,,,,,,,...,,,,,,,,Ym_ZEB1_ZFPM2,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,,,,,,,,,,,...,,,,,,,,,,
6197,,,,,,,,,,,...,,,,,,,,,,
6198,,,,,,,,,,,...,,,,,,,,,,
6199,,,,,,,,,,,...,,,,,,,,,,


In [278]:
total

,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA3,ABCA4,ABCA8,ABCA9,ABCB1,...,m_USP8_UTP20,m_VAV2_VCAN,m_VWF_WDR33,m_WFS1_WNK4,m_WSB1_WWC1,m_YBX2_YTHDC1,m_ZBTB17_ZC3H12A,m_ZEB1_ZFPM2,m_ZMIZ1_ZNF185,m_ZPBP_ZYX
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,Rm_VWF_WDR33,,,,,,,
3,,,,,,,,,,,...,,,,,,,,Ym_ZEB1_ZFPM2,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8742,,,,,,,,,TABCA9,,...,,,,,,,,,,
8743,,,,,,,,,,,...,,,,,,,,,,
8744,,,,,LABCA1,RABCA3,GABCA4,PABCA8,PABCA9,,...,Pm_USP8_UTP20,Pm_VAV2_VCAN,Cm_VWF_WDR33,Pm_WFS1_WNK4,Wm_WSB1_WWC1,Nm_YBX2_YTHDC1,,Em_ZEB1_ZFPM2,Dm_ZMIZ1_ZNF185,Nm_ZPBP_ZYX
8745,,,,,,,RABCA4,,,,...,,,,,,,,,,


# 데이터 변환 1차완료

In [5]:
expert = pd.read_csv('../reference_data/BRCA.csv')
asso =pd.read_csv('../expert_r/BRCA-associ.csv')

upg = pd.read_csv('../expert_r/BRCA-upregulated.csv')
tup = set(upg['g.name'])
downg = pd.read_csv('../expert_r/BRCA-down.csv')
tdown = set(downg['g.name'])


In [ ]:

#for d in disease_list : 

tdf = df[df['SUBCLASS']=='BRCA']
td= tdf.apply(lambda x : x!='WT').agg(sum).reset_index(name='count').sort_values(by='count', ascending=False)
#print(len(td[td['count']>0]), tdf.shape[0])

t1 = set(td[td['count']>0][2:]['index'])

#td[td['count']>5]
td[td['count']>0][1:]#['index']


In [ ]:
df_test['COL12A1'].value_counts()

In [ ]:
df_test[df_test['S100A12']!='']

In [77]:
df['COL12A1'].value_counts().reset_index().sort_values(by='COL12A1', ascending=False).to_csv('col12a1.csv')

In [ ]:
t1.intersection(set(asso['g.name']))

In [ ]:
#하위개수 1500
len(set(td[td['count']>0][1500:]['index']).intersection(tup))

In [ ]:
len(set(td[td['count']>0][1500:]['index']).intersection(tdown))

In [ ]:
len(set(td[td['count']>0][1500:]['index']).intersection(set(asso['g.name'])))

In [ ]:
len(set(td[td['count']>0][1500:]['index']).intersection(set(expert['g.name'])))

In [ ]:
#상위 개수
set(td[td['count']>0][2:1500]['index']).intersection(tup)

In [ ]:
set(td[td['count']>0][2:1500]['index']).intersection(tdown)

In [ ]:
set(td[td['count']>0][1:1500]['index']).intersection(set(asso['g.name']))

In [ ]:
set(td[td['count']>0][2:1500]['index']).intersection(set(expert['g.name']))

In [ ]:

#for d in disease_list : 

tdf = df[df['SUBCLASS']=='PAAD']
td= tdf.apply(lambda x : x!='WT').agg(sum).reset_index(name='count').sort_values(by='count', ascending=False)
#print(len(td[td['count']>0]), tdf.shape[0])

t1 = set(td[td['count']>0][2:]['index'])

#td[td['count']>5]
td[td['count']>0][2:]#['index']


In [ ]:
len(t1.intersection(tup))

In [ ]:
len(t1.intersection(tdown))

In [ ]:
len(t1.intersection(set(asso['g.name'])))

In [ ]:
len(t1.intersection(set(expert['g.name'])))

In [ ]:
df[df.SUBCLASS=='BRCA']['RNF213'].apply(lambda x : x!='WT').agg(sum)

In [ ]:
df[df.SUBCLASS=='BRCA']['AHNAK'].apply(lambda x : x!='WT').agg(sum)

In [ ]:
df[df.SUBCLASS=='BRCA']['DMD'].apply(lambda x : x!='WT').agg(sum)

In [ ]:
df[df.SUBCLASS=='BRCA']['DST'].apply(lambda x : x!='WT').agg(sum)

In [ ]:
t1.intersection(set(expert['g.name']))

In [ ]:
t1.intersection(set(asso['g.name']))

In [ ]:

for g in asso['g.name']:
    try :
        print(df[df.SUBCLASS=='BRCA'][g].apply(lambda x : x!='WT').agg(sum))
    except KeyError:
        print(g)

In [ ]:
data

In [ ]:
df_test

In [5]:
#1. train data에서 모두 WT인 거 삭제하기 
#td = data.apply(lambda x : x=='WT').agg(sum , axis=1).sort_values(ascending=False) == data.shape[1]
#df.drop(list(td[td].index), inplace=True)

#data = df.drop(columns=['ID', 'SUBCLASS'])


In [6]:
#df.iloc[td[td].index].SUBCLASS.value_counts()

In [7]:
#too many 변이, ROWs DELETE
#for d in disease_list:
#    td = df[df['SUBCLASS']==d].apply(lambda x : x!='WT').agg(sum , axis=1).sort_values(ascending=False) >= 800 #706
#    if td[td].sum()> 0:
#        print(d, list(td[td].index))
#        df.drop(list(td[td].index), inplace=True)


# Feature 선택을 위한 데이터 2차변환

### y target값만 LabelEncoding

In [279]:
y = df['SUBCLASS']
encoder2 = LabelEncoder()
y = encoder2.fit_transform(y)

In [280]:
total

,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA3,ABCA4,ABCA8,ABCA9,ABCB1,...,m_USP8_UTP20,m_VAV2_VCAN,m_VWF_WDR33,m_WFS1_WNK4,m_WSB1_WWC1,m_YBX2_YTHDC1,m_ZBTB17_ZC3H12A,m_ZEB1_ZFPM2,m_ZMIZ1_ZNF185,m_ZPBP_ZYX
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,Rm_VWF_WDR33,,,,,,,
3,,,,,,,,,,,...,,,,,,,,Ym_ZEB1_ZFPM2,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8742,,,,,,,,,TABCA9,,...,,,,,,,,,,
8743,,,,,,,,,,,...,,,,,,,,,,
8744,,,,,LABCA1,RABCA3,GABCA4,PABCA8,PABCA9,,...,Pm_USP8_UTP20,Pm_VAV2_VCAN,Cm_VWF_WDR33,Pm_WFS1_WNK4,Wm_WSB1_WWC1,Nm_YBX2_YTHDC1,,Em_ZEB1_ZFPM2,Dm_ZMIZ1_ZNF185,Nm_ZPBP_ZYX
8745,,,,,,,RABCA4,,,,...,,,,,,,,,,


### 자체 라벨링 Label Encoder XX

In [281]:
# 데이터프레임을 셀 단위로 고유값으로 변환
unique_values, unique_indices = pd.factorize(total.values.ravel())

# 고유값을 다시 데이터프레임 모양으로 변환
total_transformed = pd.DataFrame(unique_values.reshape(total.shape), columns=total.columns)

print(total_transformed)

      AAAS  AADAT  AARS1  ABAT  ABCA1  ABCA3  ABCA4  ABCA8  ABCA9  ABCB1  ...  \
0        0      0      0     0      0      0      0      0      0      0  ...   
1        0      0      0     0      0      0      0      0      0      0  ...   
2        0      0      0     0      0      0      0      0      0      0  ...   
3        0      0      0     0      0      0      0      0      0      0  ...   
4        0      0      0     0      0      0      0      0      0      0  ...   
...    ...    ...    ...   ...    ...    ...    ...    ...    ...    ...  ...   
8742     0      0      0     0      0      0      0      0  20524      0  ...   
8743     0      0      0     0      0      0      0      0      0      0  ...   
8744     0      0      0     0   7353   5746   5302   2220  19367      0  ...   
8745     0      0      0     0      0      0   7679      0      0      0  ...   
8746     0      0      0     0      0      0      0      0      0      0  ...   

      m_USP8_UTP20  m_VAV2_

In [282]:
total_transformed

,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA3,ABCA4,ABCA8,ABCA9,ABCB1,...,m_USP8_UTP20,m_VAV2_VCAN,m_VWF_WDR33,m_WFS1_WNK4,m_WSB1_WWC1,m_YBX2_YTHDC1,m_ZBTB17_ZC3H12A,m_ZEB1_ZFPM2,m_ZMIZ1_ZNF185,m_ZPBP_ZYX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,159,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,166,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8742,0,0,0,0,0,0,0,0,20524,0,...,0,0,0,0,0,0,0,0,0,0
8743,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8744,0,0,0,0,7353,5746,5302,2220,19367,0,...,3893,4518,31418,5416,48026,42651,0,6020,5088,40325
8745,0,0,0,0,0,0,7679,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Option1) Feature selection - method with XGB


In [312]:
nRows = df.shape[0]
X = total_transformed[:nRows]
df_test = total_transformed[nRows:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=156)  

### 2000개 feature 선택

In [326]:
from sklearn.metrics import accuracy_score
# Step 5: Train an XGBoost model to find important features
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# Get feature importances
feature_importances = xgb_model.feature_importances_
sorted_idx = feature_importances.argsort()

# Select top 2000 features based on importance
top_n = 1800  # Adjust this value to select the top N important features
top_features_idx = sorted_idx[-top_n:]

# Step 6: Reduce training and testing data to the top N features
X_train_reduced = X_train.iloc[:, top_features_idx]
X_test_reduced = X_test.iloc[:, top_features_idx]

# Step 7: Train a new XGBoost model on the reduced feature set
best_xgb = xgb.XGBClassifier(n_estimators=100, random_state=42)
best_xgb.fit(X_train_reduced, y_train)

# Step 8: Evaluate the model on the test set
y_pred = best_xgb.predict(X_test_reduced)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the reduced feature set: {accuracy}")

Accuracy on the reduced feature set: 0.3401397098334229


In [ ]:
#0.32
#0.34
#0.342


In [327]:
len(top_features_idx)

1800

In [328]:
total_transformed2 = total_transformed.iloc[:, top_features_idx]

## ====##  # 선택된 Feature, total_transformed2에 적용 완료


In [18]:
total_transformed2 = total_transformed.loc[:, list(glist)]

In [19]:
nRows = df.shape[0]
X = total_transformed2[:nRows]
df_test = total_transformed2[nRows:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=156)  

In [ ]:
X_train

### option2 끝

# ====

In [20]:
X_test_final = df_test.iloc[:, top_features_idx]

NameError: name 'top_features_idx' is not defined

In [ ]:
y_test_pred = best_xgb.predict(X_test_final)

In [ ]:
y_test_pred

In [ ]:

# Step 11: Decode the predicted class labels back to their original subclass names
y_test_pred_labels = encoder2.inverse_transform(y_test_pred)
res_df0 = pd.DataFrame()
res_df0['ID'] = test_ids
# Step 12: Store the predictions in the test DataFrame
res_df0['SUBCLASS'] = y_test_pred_labels

# Display the first few rows of predictions
res_df0[['SUBCLASS']].value_counts()


In [33]:
res_df0[['ID', 'SUBCLASS']].to_csv('./XGB_FEATURESELECTION.csv',index=False)


# ====

# DMatrix, XGBClassifier 적용

In [329]:
nRows = df.shape[0]
X = total_transformed2[:nRows]
df_test = total_transformed2[nRows:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=156) #shuffle=False,stratify = y, 
print(X_train.shape, X_test.shape)

(4340, 1800) (1861, 1800)


In [314]:
nRows = df.shape[0]
X = total_transformed[:nRows]
df_test = total_transformed[nRows:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=156) #shuffle=False,stratify = y, 
print(X_train.shape, X_test.shape)

(4340, 4015) (1861, 4015)


In [330]:
dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)

In [331]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    'num_class': 26
}
num_boost_round = 50

dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)
model = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_boost_round , early_stopping_rounds=5, 
                  evals=[(dtrain, 'train'), (dtest, 'valid')],
                  verbose_eval=2)

[0]	train-mlogloss:2.61417	valid-mlogloss:2.70332
[2]	train-mlogloss:2.21663	valid-mlogloss:2.45020
[4]	train-mlogloss:1.98800	valid-mlogloss:2.33097
[6]	train-mlogloss:1.82517	valid-mlogloss:2.25870
[8]	train-mlogloss:1.70558	valid-mlogloss:2.21503
[10]	train-mlogloss:1.61149	valid-mlogloss:2.18249
[12]	train-mlogloss:1.53498	valid-mlogloss:2.16203
[14]	train-mlogloss:1.47263	valid-mlogloss:2.14916
[16]	train-mlogloss:1.41744	valid-mlogloss:2.13665
[18]	train-mlogloss:1.37087	valid-mlogloss:2.12744
[20]	train-mlogloss:1.32985	valid-mlogloss:2.12409
[22]	train-mlogloss:1.29284	valid-mlogloss:2.11882
[24]	train-mlogloss:1.25845	valid-mlogloss:2.11886
[26]	train-mlogloss:1.22794	valid-mlogloss:2.11720
[28]	train-mlogloss:1.19916	valid-mlogloss:2.11830
[30]	train-mlogloss:1.17252	valid-mlogloss:2.11756
[31]	train-mlogloss:1.15979	valid-mlogloss:2.11734


In [353]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    #'learning_rate': 0.05
}
num_boost_round = 1000

clf = xgb.XGBClassifier( n_estimators = 200,        # 부스팅 단계 수
                        max_depth = 6,             # 개별 트리의 최대 깊이
                        learning_rate = 0.1,       # 학습률, 낮출수록 느리게 학습하지만 성능 개선 가능
                        subsample=0.7,           # 각 부스팅 단계에서 사용할 샘플의 비율
                        #scale_pos_weight = class_weight_dict,  # 클래스별 가중치
                        colsample_bytree=0.3,    # 각 트리에서 사용할 특성의 비율
                        random_state = 42,         # 랜덤 시드
                        n_jobs = -1,             # 모든 CPU 코어 사용,
                        reg_alpha=0.03,
                        early_stopping_rounds = 50,                        
                        **params)
clf.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test,y_test)],
        verbose=2);

[0]	validation_0-mlogloss:3.14951	validation_1-mlogloss:3.16471
[2]	validation_0-mlogloss:2.97485	validation_1-mlogloss:3.03442
[4]	validation_0-mlogloss:2.85261	validation_1-mlogloss:2.95277
[6]	validation_0-mlogloss:2.73700	validation_1-mlogloss:2.86546
[8]	validation_0-mlogloss:2.64497	validation_1-mlogloss:2.80193
[10]	validation_0-mlogloss:2.54865	validation_1-mlogloss:2.72778
[12]	validation_0-mlogloss:2.45071	validation_1-mlogloss:2.66049
[14]	validation_0-mlogloss:2.37443	validation_1-mlogloss:2.60764
[16]	validation_0-mlogloss:2.31561	validation_1-mlogloss:2.57487
[18]	validation_0-mlogloss:2.25739	validation_1-mlogloss:2.53887
[20]	validation_0-mlogloss:2.18884	validation_1-mlogloss:2.48901
[22]	validation_0-mlogloss:2.13747	validation_1-mlogloss:2.45867
[24]	validation_0-mlogloss:2.08503	validation_1-mlogloss:2.42473
[26]	validation_0-mlogloss:2.04289	validation_1-mlogloss:2.39896
[28]	validation_0-mlogloss:2.00429	validation_1-mlogloss:2.37989
[30]	validation_0-mlogloss:1.9

In [354]:
y_true = y_test
y_pred = clf.predict(X_test)
y_score = clf.predict_proba(X_test)
y_true.shape, y_pred.shape, y_score.shape

((1861,), (1861,), (1861, 26))

In [355]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

0.35626007522837183

In [344]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

0.34981192907039227

In [356]:
metrics.roc_auc_score(y_true, y_score, average='weighted', multi_class='ovr')

0.8686821296731695

In [335]:
metrics.roc_auc_score(y_true, y_score, average='weighted', multi_class='ovr')

0.8565520525429748

In [357]:
clf.predict(df_test)

array([21, 22,  2, ...,  3, 10, 11])

In [358]:
res = clf.predict(df_test)
res = encoder2.inverse_transform(res)

df_test1 = pd.read_csv('../input/test.csv')
res_df = pd.DataFrame ({'ID' : df_test1['ID'], 'SUBCLASS' : res})
res_df


,ID,SUBCLASS
0,TEST_0000,STES
1,TEST_0001,TGCT
2,TEST_0002,BRCA
3,TEST_0003,LGG
4,TEST_0004,LUSC
...,...,...
2541,TEST_2541,COAD
2542,TEST_2542,PAAD
2543,TEST_2543,CESC
2544,TEST_2544,LAML


In [359]:
res_df.SUBCLASS.value_counts()

SUBCLASS
BRCA      613
COAD      441
KIPAN     249
STES      204
GBMLGG    114
OV        111
HNSC       90
UCEC       74
KIRC       71
THCA       68
SKCM       67
LGG        58
TGCT       46
CESC       45
PRAD       42
LIHC       37
PCPG       34
PAAD       31
LAML       29
SARC       28
BLCA       26
LUSC       21
ACC        19
LUAD       19
THYM        7
DLBC        2
Name: count, dtype: int64

In [360]:
res_df.to_csv('res-1800select-01.csv',index = False)

# 1차 결과 저장 완료

In [75]:
selected_genes = set()
for d in disease_list:
    tdf = df [ df['SUBCLASS'].isin([d] )]
    total_len = tdf.shape[0]
    
    tdf = tdf.apply(lambda x : x !='WT').agg(sum)
    tdf = tdf.iloc[2:].reset_index(name='count').sort_values(by='count', ascending=False)
    #thr[d]
    rlist =[]
    if total_len > 200: 
        rlist = tdf.loc[tdf['count'] > 5]['index'].to_list()
    else :
        rlist = tdf.loc[tdf['count'] > 5]['index'].to_list()

    print(d, len(rlist))
    selected_genes.update(rlist)
selected_genes = list(selected_genes)
len(selected_genes)

KIPAN 285
SARC 31
SKCM 2112
KIRC 68
GBMLGG 145
STES 2034
BRCA 691
LIHC 106
HNSC 334
PAAD 22
OV 47
PRAD 16
UCEC 2006
LAML 9
COAD 936
THCA 3
ACC 51
LGG 16
LUSC 587
LUAD 468
CESC 175
PCPG 3
BLCA 209
TGCT 14
THYM 1
DLBC 17


2925

In [8]:
def get_gene_list1():
    selected_genes = set()
    for d in disease_list:
        count_df = df[df.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum)
        count_df = count_df[2:].reset_index(name='count')        
        gene_list = count_df[count_df['count']>5]['index'].values
        print(d, len(gene_list))
        selected_genes.update(gene_list)
    print(len(selected_genes))
    return selected_genes
glist = get_gene_list1()

KIPAN 286
SARC 31
SKCM 2112
KIRC 68
GBMLGG 147
STES 2034
BRCA 691
THCA 4
LIHC 106
HNSC 334
PAAD 22
OV 47
PRAD 16
UCEC 2006
LAML 9
COAD 936
ACC 51
LGG 16
LUSC 587
LUAD 468
CESC 175
PCPG 3
THYM 1
BLCA 209
TGCT 14
DLBC 17
2925


In [128]:
if 'SUBCLASS' in selected_genes:
    print('ry')

In [129]:

data = data.loc[:, list(selected_genes)]
df_test = df_test.loc[:, list(selected_genes)]


#Multi-class classification

## 01 Update Result with Zeros []

In [361]:
#zerosl = list(df_test[(df_test_2 == 0).all(axis=1)].index)
zerosl = [17, 180, 187, 214, 219, 294, 374, 446, 450, 545, 676, 696, 787, 855, 979, 1055, 1249, 1542, 1770, 1785, 1832, 1866, 1965, 2046, 2059, 2118, 2132, 2153, 2340, 2412, 2481]


In [362]:
res_df.loc[zerosl, 'SUBCLASS'] = 'THYM'

In [363]:
res_df

,ID,SUBCLASS
0,TEST_0000,STES
1,TEST_0001,TGCT
2,TEST_0002,BRCA
3,TEST_0003,LGG
4,TEST_0004,LUSC
...,...,...
2541,TEST_2541,COAD
2542,TEST_2542,PAAD
2543,TEST_2543,CESC
2544,TEST_2544,LAML


In [364]:
res_df.SUBCLASS.value_counts()

SUBCLASS
BRCA      582
COAD      441
KIPAN     249
STES      204
GBMLGG    114
OV        111
HNSC       90
UCEC       74
KIRC       71
THCA       68
SKCM       67
LGG        58
TGCT       46
CESC       45
PRAD       42
THYM       38
LIHC       37
PCPG       34
PAAD       31
LAML       29
SARC       28
BLCA       26
LUSC       21
ACC        19
LUAD       19
DLBC        2
Name: count, dtype: int64

In [366]:
res_df.loc[180]

ID          TEST_0180
SUBCLASS         THYM
Name: 180, dtype: object

## SAVE res_df with zero step

In [365]:
res_df.to_csv('../res2000-selected-step-1.csv',index = False)

# Second Step

In [91]:
import pandas as pd

In [374]:

df = pd.read_csv('../input/train.csv')
df_test1 = pd.read_csv('../input/test.csv')

data1 = df.drop(columns=['ID', 'SUBCLASS'])
df_test1.drop(columns='ID',inplace=True)

total3 = pd.concat([data1, df_test1])

f = lambda x : 0 if x == 'WT' else 1
total3 = total3.map(f)

ns = df.shape[0]
data3 = total3[:ns]
df_test1 = total3[ns:]


In [375]:
df_test1

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2543,0,0,0,0,0,1,1,1,1,1,...,1,1,0,0,1,0,1,1,0,0
2544,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [379]:
data3

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [376]:
# 각 행에서 1이 위치한 열의 이름과 인덱스를 구하는 코드
one_1_rows = df_test1[df_test1.sum(axis=1) == 1]
result = [(index, row.idxmax()) for index, row in one_1_rows.iterrows()]
result

[(8, 'SOX9'),
 (90, 'BRAF'),
 (110, 'LIG1'),
 (131, 'IDH2'),
 (170, 'PLK4'),
 (192, 'ZNF365'),
 (260, 'BRAF'),
 (352, 'PRKDC'),
 (366, 'VHL'),
 (390, 'DOCK2'),
 (435, 'CDCP1'),
 (619, 'GFPT2'),
 (655, 'CTNNA1'),
 (672, 'CD46'),
 (693, 'HRAS'),
 (701, 'HRAS'),
 (798, 'CCNL1'),
 (880, 'GNAO1'),
 (898, 'ATXN1'),
 (1023, 'MYH1'),
 (1034, 'CDKN2A'),
 (1071, 'VHL'),
 (1078, 'CACNA1A'),
 (1118, 'ACE'),
 (1202, 'VHL'),
 (1263, 'BRAF'),
 (1287, 'TG'),
 (1408, 'VHL'),
 (1426, 'VHL'),
 (1437, 'SREBF2'),
 (1560, 'CACNA1A'),
 (1690, 'LRPPRC'),
 (1761, 'VHL'),
 (1804, 'RAD21'),
 (1830, 'LIG1'),
 (1833, 'VHL'),
 (1851, 'NPM1'),
 (1879, 'VHL'),
 (1884, 'KMT2D'),
 (1979, 'NPM1'),
 (2056, 'VHL'),
 (2086, 'VHL'),
 (2114, 'BRCA2'),
 (2150, 'NPM1'),
 (2218, 'TP53'),
 (2383, 'NPM1'),
 (2447, 'CDH1'),
 (2512, 'SUCLG1')]

In [380]:
one_VHL_l=[]
one_BRAF_l = []
one_NPM1_l = []
one_LIG1_l = []
one_s = []
res_df1 = res_df.copy()

for i, c in result:
    print(i, c)
    if c =='VHL': 
        one_VHL_l.append(i)
        # 해당 컬럼에서 SUBCLASS값들의 분포를 구함
        q = ['VHL']
        rows_with_all_ones = df[data3[q].eq(1).all(axis=1)] 
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS #KIRC
        res_df1.loc[one_VHL_l,'SUBCLASS'] = rs
    elif c=='BRAF': 
        one_BRAF_l.append(i)
        q = ['BRAF']
        rows_with_all_ones = df[data3[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS #maybe 1
        res_df1.loc[one_BRAF_l,'SUBCLASS'] = rs
    elif c=='NPM1' : 
        one_NPM1_l.append(i)
        q = ['NPM1']
        rows_with_all_ones = df[data3[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS#0 not 1
        res_df1.loc[one_NPM1_l,'SUBCLASS'] = rs
    elif c=='LIG1' : 
        one_LIG1_l.append(i)
        q = ['LIG1']
        rows_with_all_ones = df[data3[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS#0 not 1
        res_df1.loc[one_LIG1_l,'SUBCLASS'] = rs
    elif c=='HRAS': 
        one_s.append(i)
        q = ['HRAS']
        rows_with_all_ones = df[data3[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS#0 not 1
        res_df1.loc[one_s,'SUBCLASS'] = rs
    #else:
        ###one_s.append(i)
        #row_no = i
        #q = [c]
        #rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        #rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS
        #res_df1.loc[i]['SUBCLASS'] = rs
    

res_df1.to_csv('20241013-step2-ones.csv',index = False)
    

8 SOX9
90 BRAF
110 LIG1
131 IDH2
170 PLK4
192 ZNF365
260 BRAF
352 PRKDC
366 VHL
390 DOCK2
435 CDCP1
619 GFPT2
655 CTNNA1
672 CD46
693 HRAS
701 HRAS
798 CCNL1
880 GNAO1
898 ATXN1
1023 MYH1
1034 CDKN2A
1071 VHL
1078 CACNA1A
1118 ACE
1202 VHL
1263 BRAF
1287 TG
1408 VHL
1426 VHL
1437 SREBF2
1560 CACNA1A
1690 LRPPRC
1761 VHL
1804 RAD21
1830 LIG1
1833 VHL
1851 NPM1
1879 VHL
1884 KMT2D
1979 NPM1
2056 VHL
2086 VHL
2114 BRCA2
2150 NPM1
2218 TP53
2383 NPM1
2447 CDH1
2512 SUCLG1


In [132]:
############# optional code

In [381]:
## TEST 데이터에서 1개의 컬럼만 1인 경우 찾기
one_1_rows = df_test1[df_test1.sum(axis=1) == 1]
# 각 열(column)별로 1이 있는 인덱스 리스트 생성
result_dict = {}
for column in one_1_rows.columns:
    indices = one_1_rows[one_1_rows[column] == 1].index.tolist()
    if indices:  # 인덱스가 비어있지 않으면 추가
        result_dict[column] = indices

# 딕셔너리를 DataFrame으로 변환
result_df = pd.DataFrame(list(result_dict.items()), columns=['gene', 'index_list'])
result_df


,gene,index_list
0,ACE,[1118]
1,ATXN1,[898]
2,BRAF,"[90, 260, 1263]"
3,BRCA2,[2114]
4,CACNA1A,"[1078, 1560]"
5,CCNL1,[798]
6,CD46,[672]
7,CDCP1,[435]
8,CDH1,[2447]
9,CDKN2A,[1034]


In [ ]:
q = 'VHL'
rows_with_all_ones = df[(data[q] == 1) & (data.drop(columns=q) == 0).all(axis=1)]
rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
#res_df1.loc[one_VHL_l,'SUBCLASS'] = rs
print(rs)

In [ ]:
## df_test 데이터에서 2개의 컬럼만 1인 경우 찾기
one_2_rows = df_test[df_test.sum(axis=1) == 2]
# 각 행에서 1이 위치한 열의 이름을 나열
result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]

# 리스트를 DataFrame으로 변환
result_df = pd.DataFrame(result, columns=['index', 'genes'])
result_df



In [ ]:
## df_test 데이터에서 2개의 컬럼만 1인 경우 찾기
one_2_rows = df_test[df_test.sum(axis=1) == 2]
# 각 행에서 1이 위치한 열의 이름을 나열
result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]

# 리스트를 DataFrame으로 변환
result_df = pd.DataFrame(result, columns=['index', 'genes'])
result_df



In [ ]:
## 1이 2개 이상 있는 행 필터링
one_2_rows = df_test[df_test.sum(axis=1) == 2]

# 각 행에서 1이 위치한 열의 이름을 모두 나열하는 코드
result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]

# 출력
print(result)


## Update Res_df with 2 / 3 / 4 / 5

In [382]:
## TEST DATA에서 1이 n개 있는 행 필터링
def find_test_n_ones(n):
    one_2_rows = df_test1[df_test1.sum(axis=1) == n]
    # 각 행에서 1이 위치한 열의 이름을 모두 나열하는 코드
    result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]
    print(len(result))
    return result
print(find_test_n_ones(2))

47
[(74, ['CDH8', 'RYR2']), (75, ['HKDC1', 'UGCG']), (94, ['TGFBRAP1', 'TMPRSS6']), (100, ['DPYD', 'SYMPK']), (139, ['AADAT', 'JUP']), (384, ['DDX10', 'ELOVL5']), (387, ['CUL4A', 'ZFPM2']), (409, ['ARFGEF2', 'BRAF']), (534, ['CCR4', 'TLR7']), (565, ['BMP1', 'BRAF']), (637, ['CHEK2', 'UBC']), (733, ['BRAF', 'MXRA5']), (757, ['DCC', 'LRP1']), (772, ['CDON', 'MAP4K1']), (780, ['BRAF', 'LAMA4']), (796, ['NPM1', 'TTC39A']), (869, ['CCL25', 'HELZ2']), (878, ['PYGM', 'TGFBRAP1']), (899, ['ENPP1', 'SPRED2']), (911, ['IL1A', 'VWF']), (1042, ['ADAM23', 'RYR2']), (1065, ['MXD4', 'TCHH']), (1093, ['CPA2', 'DTX2']), (1267, ['CBR1', 'SLC19A1']), (1388, ['BRAF', 'LSP1']), (1390, ['EGLN2', 'NPM1']), (1520, ['ITIH1', 'PLG']), (1532, ['CDR2', 'XIAP']), (1555, ['HLA-DRB1', 'PREX1']), (1567, ['CTNS', 'IL12RB1']), (1571, ['DIO2', 'NPM1']), (1622, ['PROP1', 'ZNFX1']), (1634, ['SLIT1', 'ST6GAL1']), (1639, ['IRS4', 'LCP2']), (1756, ['COL12A1', 'TRIP10']), (1779, ['TGFBRAP1', 'TMPRSS6']), (1808, ['FGA', 'NIPBL

In [ ]:
df.iloc[5618].apply(lambda x : x!='WT').agg(sum)

In [ ]:
df.iloc[1954].SUBCLASS

In [383]:
# exact matching은 보정
res_df2 = res_df1.copy()
for ns in range (1, 5):
    result_n = find_test_n_ones(ns)

    for i, qSet in result_n:
        # qSet에 해당하는 컬럼들만 값이 1인 행을 반환
        rows_with_only_qSet_ones = df[(data3[qSet].eq(1).all(axis=1)) & (data3.drop(columns=qSet).eq(0).all(axis=1))]
        rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
        
        if rs.shape[0] == 1:
            rs = rs.loc[0].SUBCLASS
            if rs != res_df.loc[i]['SUBCLASS']:
                print(ns, rs, res_df1.loc[i]['SUBCLASS'], qSet)
                res_df2.loc[i,'SUBCLASS'] = rs
res_df2.to_csv('s20241013-step3-exactMatching.csv',index = False)


48
1 THCA BRCA ['PRKDC']
1 LAML BRCA ['RAD21']
1 KIRC KIPAN ['KMT2D']
1 KIPAN BRCA ['SUCLG1']
47
2 LGG BRCA ['CTNS', 'IL12RB1']
60
3 KIRC GBMLGG ['MMP2', 'PTEN', 'TRIM58']
3 KIPAN PCPG ['CHEK2', 'PIK3CG', 'TAF1C']
3 KIPAN BRCA ['CIITA', 'PKHD1', 'RAPGEF6']
3 GBMLGG LGG ['ATRX', 'GM2A', 'IDH1']
65
4 GBMLGG LGG ['IDH1', 'LATS2', 'MYH8', 'PDGFA']
4 LGG LAML ['ABCA4', 'ATRX', 'EXOC4', 'PTPN11']
4 LGG BRCA ['COL1A2', 'EDN2', 'KALRN', 'MXRA5']


In [384]:
res_df2.SUBCLASS.value_counts()

SUBCLASS
BRCA      574
COAD      441
KIPAN     244
STES      204
GBMLGG    115
OV        111
HNSC       90
KIRC       80
UCEC       74
SKCM       72
THCA       66
LGG        59
TGCT       46
CESC       45
PRAD       42
THYM       38
LIHC       37
PCPG       33
PAAD       31
LAML       29
SARC       28
BLCA       26
LUSC       21
ACC        19
LUAD       19
DLBC        2
Name: count, dtype: int64

In [ ]:
print(find_test_n_ones(4))

In [ ]:
print(find_test_n_ones(5))


In [ ]:
df_test

In [ ]:
# res 값 수정, 
# 문서 2,3,4,5
res_df2 = res_df1.copy()
result_n = find_test_n_ones(2)

for i, qSet in result_n:
    print(i, qSet)
    #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
    rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
    rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    
    if rs.shape[0] > 0:
        rs = rs.loc[0].SUBCLASS
        print(rs)
        res_df2.loc[i,'SUBCLASS'] = rs
    else: 
        #option1) qSet에 해당하는 컬럼들이 포함된, 값이 1인 행을 반환
        rows_with_all_ones = df[data[qSet].eq(1).all(axis=1)]
        if rows_with_all_ones.shape[0] > 1:
            try:
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count')#.loc[].SUBCLASS## 0->1
                print(rs)
                #res_df2.loc[i,'SUBCLASS'] = rs
            except KeyError or ValueError:
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS## 0->1
                res_df2.loc[i,'SUBCLASS'] = rs
                print(rs)
            
#res_df2.to_csv('res2-step3-two.csv',index = False)

In [ ]:
df

In [ ]:
result_n = find_test_n_ones(1)

for i, qSet in result_n:
    print(i, qSet)
    #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
    rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
    rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    
    if rs.shape[0] > 0:
        #rs = rs.loc[0].SUBCLASS
        print(rs)
        #res_df2.loc[i,'SUBCLASS'] = rs
    #else: 
    #    #option1) qSet에 해당하는 컬럼들이 포함된, 값이 1인 행을 반환
    #    rows_with_all_ones = df[data[qSet].eq(1).all(axis=1)]
    #    if rows_with_all_ones.shape[0] > 0 : 
    #        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS
    #        print(rs)
    #        res_df2.loc[i,'SUBCLASS'] = rs


In [55]:
res_df2.to_csv('res-step4-three.csv',index = False)

In [48]:
res_df2.to_csv('res-step5-four.csv',index = False)

In [ ]:
result_n = find_test_n_ones(5)

for i, qSet in result_n:
    print(i, qSet)
    #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
    rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
    rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    
    if rs.shape[0] > 0:
        rs = rs.loc[0].SUBCLASS
        print(rs)
        res_df2.loc[i,'SUBCLASS'] = rs
    else: 
        #option1) qSet에 해당하는 컬럼들이 포함된, 값이 1인 행을 반환
        rows_with_all_ones = df[data[qSet].eq(1).all(axis=1)]
        if rows_with_all_ones.shape[0] > 1 : 
            try: 
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS
                print(rs)
                res_df2.loc[i,'SUBCLASS'] = rs
            except KeyError or ValueError:
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS
                print(rs)
                res_df2.loc[i,'SUBCLASS'] = rs


In [58]:
res_df2.to_csv('res-step6-five.csv',index = False)